In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.stats import norm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

## 🔧 Black-Scholes Implementation

In [ ]:
def black_scholes_call(S, K, T, r, sigma):
    """
    European call option price.
    
    Args:
        S: Current stock price
        K: Strike price
        T: Time to expiration (years)
        r: Risk-free rate
        sigma: Volatility
    """
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    call_price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return call_price

def black_scholes_put(S, K, T, r, sigma):
    """
    European put option price.
    """
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    put_price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    return put_price

def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    """
    Calculate all Greeks.
    """
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        delta = norm.cdf(d1)
        theta = (-S*norm.pdf(d1)*sigma/(2*np.sqrt(T)) 
                 - r*K*np.exp(-r*T)*norm.cdf(d2))
        rho = K*T*np.exp(-r*T)*norm.cdf(d2)
    else:  # put
        delta = -norm.cdf(-d1)
        theta = (-S*norm.pdf(d1)*sigma/(2*np.sqrt(T)) 
                 + r*K*np.exp(-r*T)*norm.cdf(-d2))
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2)
    
    gamma = norm.pdf(d1) / (S*sigma*np.sqrt(T))
    vega = S*norm.pdf(d1)*np.sqrt(T)
    
    return {
        'delta': delta,
        'gamma': gamma,
        'vega': vega / 100,  # Per 1% change in vol
        'theta': theta / 365,  # Per day
        'rho': rho / 100  # Per 1% change in rate
    }

print("✅ Black-Scholes functions defined")

## 📊 Load Real-World Data: GLD-GDX Pair

In [ ]:
# Download GLD (Gold ETF) and GDX (Gold Miners ETF)
# These are known to be cointegrated

symbols = ['GLD', 'GDX']
start_date = datetime.now() - timedelta(days=730)
end_date = datetime.now()

print(f"📥 Downloading {symbols} from {start_date.date()} to {end_date.date()}...")

data = yf.download(symbols, start=start_date, end=end_date)['Adj Close']

print(f"\n✅ Downloaded {len(data)} data points")
print(f"\nLatest prices:")
print(data.tail())

In [ ]:
# Calculate spread and basic stats
from statsmodels.tsa.stattools import coint

# Cointegration test
score, p_value, _ = coint(data['GLD'], data['GDX'])

# Calculate hedge ratio
beta = np.polyfit(data['GDX'], data['GLD'], 1)[0]
spread = data['GLD'] - beta * data['GDX']

# Calculate returns for volatility
returns_gld = data['GLD'].pct_change().dropna()
returns_gdx = data['GDX'].pct_change().dropna()

vol_gld = returns_gld.std() * np.sqrt(252)
vol_gdx = returns_gdx.std() * np.sqrt(252)

print(f"\n📊 Pair Statistics:\n")
print(f"Cointegration p-value: {p_value:.4f}")
print(f"Hedge ratio (beta): {beta:.4f}")
print(f"\nGLD:")
print(f"  Current Price: ${data['GLD'].iloc[-1]:.2f}")
print(f"  Annual Volatility: {vol_gld:.2%}")
print(f"\nGDX:")
print(f"  Current Price: ${data['GDX'].iloc[-1]:.2f}")
print(f"  Annual Volatility: {vol_gdx:.2%}")
print(f"\nSpread:")
print(f"  Current: {spread.iloc[-1]:.2f}")
print(f"  Mean: {spread.mean():.2f}")
print(f"  Std Dev: {spread.std():.2f}")

In [ ]:
# Visualize the pair
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Normalized prices
norm_gld = data['GLD'] / data['GLD'].iloc[0] * 100
norm_gdx = data['GDX'] / data['GDX'].iloc[0] * 100

axes[0].plot(norm_gld, label='GLD (Gold ETF)', linewidth=2)
axes[0].plot(norm_gdx, label='GDX (Gold Miners ETF)', linewidth=2)
axes[0].set_ylabel('Normalized Price')
axes[0].set_title('GLD vs GDX: Normalized Price Movement')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Spread
axes[1].plot(spread, label='Spread (GLD - β×GDX)', color='purple', linewidth=2)
axes[1].axhline(spread.mean(), color='black', linestyle='--', label='Mean')
axes[1].axhline(spread.mean() + spread.std(), color='red', linestyle='--', label='±1σ', alpha=0.7)
axes[1].axhline(spread.mean() - spread.std(), color='red', linestyle='--', alpha=0.7)
axes[1].fill_between(spread.index, spread.mean() - spread.std(), 
                     spread.mean() + spread.std(), alpha=0.1, color='gray')
axes[1].set_ylabel('Spread')
axes[1].set_xlabel('Date')
axes[1].set_title(f'Cointegrated Spread (p-value: {p_value:.4f})')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 Strategy 1: Long Call for Leverage

### Strategy Description
When spread crosses below mean (buy signal), instead of buying the spread directly:
- Buy **ATM call option** on GLD
- Short GDX shares (hedge ratio)

### Advantages
- ✅ Limited downside risk (premium paid)
- ✅ Leveraged upside exposure
- ✅ Lower capital requirement

### Risks
- ❌ Time decay (theta)
- ❌ Premium cost
- ❌ Volatility risk (vega)

In [ ]:
# Strategy parameters
S_gld = data['GLD'].iloc[-1]  # Current GLD price
K_call = S_gld  # ATM strike
T = 30 / 365  # 30 days to expiration
r = 0.04  # Risk-free rate
sigma_gld = vol_gld  # Historical volatility

# Calculate call option price
call_price = black_scholes_call(S_gld, K_call, T, r, sigma_gld)
greeks = calculate_greeks(S_gld, K_call, T, r, sigma_gld, 'call')

print(f"\n📈 Long Call Strategy (GLD):\n")
print(f"Current GLD Price: ${S_gld:.2f}")
print(f"Strike Price: ${K_call:.2f} (ATM)")
print(f"Time to Expiration: 30 days")
print(f"Implied Volatility: {sigma_gld:.2%}")
print(f"\nOption Price: ${call_price:.2f}")
print(f"\nThe Greeks:")
print(f"  Delta: {greeks['delta']:.4f} (moves ${greeks['delta']:.2f} per $1 move in GLD)")
print(f"  Gamma: {greeks['gamma']:.4f} (delta changes by {greeks['gamma']:.4f} per $1 move)")
print(f"  Vega: ${greeks['vega']:.2f} (per 1% change in volatility)")
print(f"  Theta: ${greeks['theta']:.2f} (per day time decay)")
print(f"  Rho: ${greeks['rho']:.2f} (per 1% change in interest rate)")

# Calculate breakeven
breakeven = K_call + call_price
print(f"\nBreakeven at Expiration: ${breakeven:.2f}")
print(f"Required Move: {(breakeven/S_gld - 1)*100:.2f}%")

In [ ]:
# Visualize payoff diagram
spot_prices = np.linspace(S_gld * 0.8, S_gld * 1.2, 100)

# At expiration payoff
intrinsic = np.maximum(spot_prices - K_call, 0)
payoff = intrinsic - call_price

# Current payoff (with time value)
current_payoff = [black_scholes_call(S, K_call, T, r, sigma_gld) - call_price 
                  for S in spot_prices]

plt.figure(figsize=(12, 7))
plt.plot(spot_prices, payoff, label='At Expiration', linewidth=2.5, color='blue')
plt.plot(spot_prices, current_payoff, label='Current (30 days left)', 
         linewidth=2.5, linestyle='--', color='green')
plt.axhline(0, color='black', linestyle='-', linewidth=0.5)
plt.axvline(S_gld, color='red', linestyle='--', label='Current Price', alpha=0.7)
plt.axvline(breakeven, color='orange', linestyle='--', label='Breakeven', alpha=0.7)

# Fill profit/loss regions
plt.fill_between(spot_prices, 0, payoff, where=(payoff >= 0), 
                 color='green', alpha=0.2, label='Profit Zone')
plt.fill_between(spot_prices, 0, payoff, where=(payoff < 0), 
                 color='red', alpha=0.2, label='Loss Zone')

plt.xlabel('GLD Price at Expiration ($)', fontsize=12)
plt.ylabel('Profit / Loss ($)', fontsize=12)
plt.title('Long Call Payoff Diagram (GLD)', fontsize=14, fontweight='bold')
plt.legend(loc='upper left', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n💡 Interpretation:")
print(f"  Max Loss: ${call_price:.2f} (premium paid)")
print(f"  Max Profit: Unlimited")
print(f"  Profit if GLD > ${breakeven:.2f}")

## 🛡️ Strategy 2: Protective Put (Insurance)

### Strategy Description
When holding the spread (long position):
- Buy **OTM put option** on GLD
- Protects against adverse moves

### Advantages
- ✅ Downside protection
- ✅ Keeps unlimited upside
- ✅ Peace of mind

### Costs
- Premium paid (insurance cost)
- Reduces overall profitability

In [ ]:
# Protective put parameters
K_put = S_gld * 0.95  # 5% OTM put
put_price = black_scholes_put(S_gld, K_put, T, r, sigma_gld)
put_greeks = calculate_greeks(S_gld, K_put, T, r, sigma_gld, 'put')

print(f"\n🛡️ Protective Put Strategy:\n")
print(f"Current GLD Price: ${S_gld:.2f}")
print(f"Put Strike Price: ${K_put:.2f} (5% OTM)")
print(f"Put Premium: ${put_price:.2f}")
print(f"\nInsurance Cost: {(put_price/S_gld)*100:.2f}% of position value")
print(f"\nProtection Level: ${K_put:.2f}")
print(f"Max Loss (protected): ${S_gld - K_put + put_price:.2f}")
print(f"Max Loss %: {((S_gld - K_put + put_price)/S_gld)*100:.2f}%")

In [ ]:
# Combined payoff: Long stock + Long put
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

spot_prices = np.linspace(S_gld * 0.7, S_gld * 1.3, 100)

# Without protection
stock_payoff = spot_prices - S_gld

# With protective put
put_payoff = np.maximum(K_put - spot_prices, 0) - put_price
protected_payoff = stock_payoff + put_payoff

# Plot 1: Comparison
axes[0].plot(spot_prices, stock_payoff, label='Stock Only', linewidth=2.5, color='blue')
axes[0].plot(spot_prices, protected_payoff, label='Stock + Protective Put', 
             linewidth=2.5, color='green')
axes[0].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[0].axvline(S_gld, color='red', linestyle='--', label='Current Price', alpha=0.7)
axes[0].axvline(K_put, color='orange', linestyle='--', label='Put Strike', alpha=0.7)
axes[0].set_xlabel('GLD Price ($)', fontsize=12)
axes[0].set_ylabel('Profit / Loss ($)', fontsize=12)
axes[0].set_title('Protection Comparison', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Components
axes[1].plot(spot_prices, stock_payoff, label='Long Stock', linewidth=2, alpha=0.7)
axes[1].plot(spot_prices, put_payoff, label='Long Put', linewidth=2, alpha=0.7)
axes[1].plot(spot_prices, protected_payoff, label='Combined', 
             linewidth=2.5, color='green', linestyle='--')
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[1].axvline(S_gld, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('GLD Price ($)', fontsize=12)
axes[1].set_ylabel('Profit / Loss ($)', fontsize=12)
axes[1].set_title('Strategy Components', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 Key Insight:")
print(f"  The protective put creates a 'floor' at ${K_put:.2f}")
print(f"  Maximum loss is capped at ${S_gld - K_put + put_price:.2f}")
print(f"  Upside remains unlimited (minus premium)")

## 💰 Strategy 3: Covered Call (Income Generation)

### Strategy Description
When holding spread in favorable zone:
- Hold long position
- Sell **OTM call option** to collect premium

### Advantages
- ✅ Generate income (premium received)
- ✅ Reduces cost basis
- ✅ Provides downside cushion

### Trade-offs
- ❌ Caps upside potential
- ❌ Obligation to sell if called

In [ ]:
# Covered call parameters
K_covered = S_gld * 1.05  # 5% OTM call
covered_call_price = black_scholes_call(S_gld, K_covered, T, r, sigma_gld)

print(f"\n💰 Covered Call Strategy:\n")
print(f"Current GLD Price: ${S_gld:.2f}")
print(f"Call Strike (sold): ${K_covered:.2f} (5% OTM)")
print(f"Premium Received: ${covered_call_price:.2f}")
print(f"\nIncome Generation: {(covered_call_price/S_gld)*100:.2f}% of position")
print(f"Annualized Yield: {(covered_call_price/S_gld)*(365/30)*100:.2f}%")
print(f"\nNew Cost Basis: ${S_gld - covered_call_price:.2f}")
print(f"Max Profit: ${K_covered - S_gld + covered_call_price:.2f}")
print(f"Max Profit %: {((K_covered - S_gld + covered_call_price)/S_gld)*100:.2f}%")
print(f"\nUpside Capped At: ${K_covered:.2f}")

In [ ]:
# Covered call payoff
spot_prices = np.linspace(S_gld * 0.8, S_gld * 1.3, 100)

# Long stock payoff
stock_pl = spot_prices - S_gld

# Short call payoff
short_call_pl = covered_call_price - np.maximum(spot_prices - K_covered, 0)

# Combined
covered_call_pl = stock_pl + short_call_pl

plt.figure(figsize=(14, 8))
plt.plot(spot_prices, stock_pl, label='Stock Only', linewidth=2, alpha=0.6, linestyle=':')
plt.plot(spot_prices, short_call_pl, label='Short Call', linewidth=2, alpha=0.6, linestyle=':')
plt.plot(spot_prices, covered_call_pl, label='Covered Call (Combined)', 
         linewidth=3, color='darkgreen')

plt.axhline(0, color='black', linestyle='-', linewidth=0.5)
plt.axvline(S_gld, color='red', linestyle='--', label='Current Price', alpha=0.7)
plt.axvline(K_covered, color='orange', linestyle='--', label='Call Strike (Cap)', alpha=0.7)

# Highlight profit zone
plt.fill_between(spot_prices, 0, covered_call_pl, where=(covered_call_pl >= 0),
                 color='green', alpha=0.2)
plt.fill_between(spot_prices, 0, covered_call_pl, where=(covered_call_pl < 0),
                 color='red', alpha=0.2)

plt.xlabel('GLD Price at Expiration ($)', fontsize=12)
plt.ylabel('Profit / Loss ($)', fontsize=12)
plt.title('Covered Call Strategy Payoff', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n💡 Strategy Characteristics:")
print(f"  Breakeven: ${S_gld - covered_call_price:.2f}")
print(f"  Max Profit: ${max(covered_call_pl):.2f} at ${K_covered:.2f}+")
print(f"  Best for: Neutral to slightly bullish outlook")

## ⚖️ Strategy 4: Delta-Neutral Straddle

### Strategy Description
For pure mean-reversion play:
- Buy **ATM call** and **ATM put** (long straddle)
- Delta-hedge with underlying
- Profits from volatility/movement in either direction

### When to Use
- ✅ Expecting large move (high volatility)
- ✅ Uncertain about direction
- ✅ Spread far from mean (high mean-reversion probability)

### Risks
- ❌ High premium cost (2x options)
- ❌ Time decay works against you
- ❌ Need significant move to profit

In [ ]:
# Long straddle parameters
K_straddle = S_gld  # ATM
straddle_call = black_scholes_call(S_gld, K_straddle, T, r, sigma_gld)
straddle_put = black_scholes_put(S_gld, K_straddle, T, r, sigma_gld)
straddle_cost = straddle_call + straddle_put

call_greeks_straddle = calculate_greeks(S_gld, K_straddle, T, r, sigma_gld, 'call')
put_greeks_straddle = calculate_greeks(S_gld, K_straddle, T, r, sigma_gld, 'put')

net_delta = call_greeks_straddle['delta'] + put_greeks_straddle['delta']
net_gamma = call_greeks_straddle['gamma'] + put_greeks_straddle['gamma']
net_vega = call_greeks_straddle['vega'] + put_greeks_straddle['vega']
net_theta = call_greeks_straddle['theta'] + put_greeks_straddle['theta']

print(f"\n⚖️ Long Straddle (Delta-Neutral):\n")
print(f"Strike Price: ${K_straddle:.2f} (ATM)")
print(f"Call Premium: ${straddle_call:.2f}")
print(f"Put Premium: ${straddle_put:.2f}")
print(f"Total Cost: ${straddle_cost:.2f}")
print(f"\nPosition Greeks:")
print(f"  Net Delta: {net_delta:.4f} (nearly delta-neutral)")
print(f"  Net Gamma: {net_gamma:.4f} (positive convexity)")
print(f"  Net Vega: ${net_vega:.2f} (benefits from vol increase)")
print(f"  Net Theta: ${net_theta:.2f}/day (time decay cost)")
print(f"\nBreakeven Points:")
print(f"  Upper: ${K_straddle + straddle_cost:.2f} (+{(straddle_cost/S_gld)*100:.2f}%)")
print(f"  Lower: ${K_straddle - straddle_cost:.2f} (-{(straddle_cost/S_gld)*100:.2f}%)")
print(f"\nRequired Move: ±{(straddle_cost/S_gld)*100:.2f}% to breakeven")

In [ ]:
# Straddle payoff visualization
spot_prices = np.linspace(S_gld * 0.7, S_gld * 1.3, 200)

# At expiration
call_payoff_exp = np.maximum(spot_prices - K_straddle, 0)
put_payoff_exp = np.maximum(K_straddle - spot_prices, 0)
straddle_payoff = call_payoff_exp + put_payoff_exp - straddle_cost

# Current (with time value)
T_current = T * 0.5  # Halfway to expiration
current_payoff = [black_scholes_call(S, K_straddle, T_current, r, sigma_gld) +
                  black_scholes_put(S, K_straddle, T_current, r, sigma_gld) - straddle_cost
                  for S in spot_prices]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Payoff at expiration
axes[0].plot(spot_prices, straddle_payoff, linewidth=3, color='purple', label='At Expiration')
axes[0].plot(spot_prices, current_payoff, linewidth=2.5, color='green', 
             linestyle='--', alpha=0.7, label='15 days left')
axes[0].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[0].axvline(S_gld, color='red', linestyle='--', label='Current Price', alpha=0.7)
axes[0].axvline(K_straddle + straddle_cost, color='orange', linestyle=':', label='Breakeven', alpha=0.7)
axes[0].axvline(K_straddle - straddle_cost, color='orange', linestyle=':', alpha=0.7)

axes[0].fill_between(spot_prices, 0, straddle_payoff, where=(straddle_payoff >= 0),
                     color='green', alpha=0.2, label='Profit')
axes[0].fill_between(spot_prices, 0, straddle_payoff, where=(straddle_payoff < 0),
                     color='red', alpha=0.2, label='Loss')

axes[0].set_xlabel('GLD Price ($)', fontsize=12)
axes[0].set_ylabel('Profit / Loss ($)', fontsize=12)
axes[0].set_title('Long Straddle Payoff Diagram', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=9)
axes[0].grid(alpha=0.3)

# Plot 2: Greeks evolution
prices_for_greeks = np.linspace(S_gld * 0.9, S_gld * 1.1, 50)
deltas = [calculate_greeks(S, K_straddle, T, r, sigma_gld, 'call')['delta'] +
          calculate_greeks(S, K_straddle, T, r, sigma_gld, 'put')['delta']
          for S in prices_for_greeks]
gammas = [calculate_greeks(S, K_straddle, T, r, sigma_gld, 'call')['gamma'] +
          calculate_greeks(S, K_straddle, T, r, sigma_gld, 'put')['gamma']
          for S in prices_for_greeks]

ax2 = axes[1]
ax3 = ax2.twinx()

line1 = ax2.plot(prices_for_greeks, deltas, 'b-', linewidth=2, label='Net Delta')
line2 = ax3.plot(prices_for_greeks, gammas, 'r-', linewidth=2, label='Net Gamma')

ax2.axhline(0, color='black', linestyle='--', linewidth=0.5, alpha=0.5)
ax2.axvline(S_gld, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label='Current')

ax2.set_xlabel('GLD Price ($)', fontsize=12)
ax2.set_ylabel('Net Delta', color='b', fontsize=12)
ax3.set_ylabel('Net Gamma', color='r', fontsize=12)
ax2.set_title('Greeks Profile (Delta-Neutral)', fontsize=13, fontweight='bold')

ax2.tick_params(axis='y', labelcolor='b')
ax3.tick_params(axis='y', labelcolor='r')

lines = line1 + line2
labels = [l.get_label() for l in lines]
ax2.legend(lines, labels, fontsize=9)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 Straddle Insights:")
print(f"  Best Case: Large move in either direction")
print(f"  Worst Case: Price stays near ${K_straddle:.2f} (max loss: ${straddle_cost:.2f})")
print(f"  Time Decay: Loses ${abs(net_theta):.2f} per day")
print(f"  Vol Sensitivity: Gains ${net_vega:.2f} per 1% vol increase")

## 📊 Strategy Comparison Table

In [ ]:
# Create comparison DataFrame
strategies_comparison = pd.DataFrame([
    {
        'Strategy': 'Long Call',
        'Initial Cost': f"${call_price:.2f}",
        'Max Loss': f"${call_price:.2f}",
        'Max Profit': 'Unlimited',
        'Breakeven': f"${breakeven:.2f}",
        'Best For': 'Bullish, leverage',
        'Risk Level': '⭐⭐⭐'
    },
    {
        'Strategy': 'Protective Put',
        'Initial Cost': f"${S_gld + put_price:.2f}",
        'Max Loss': f"${S_gld - K_put + put_price:.2f}",
        'Max Profit': 'Unlimited',
        'Breakeven': f"${S_gld + put_price:.2f}",
        'Best For': 'Downside protection',
        'Risk Level': '⭐⭐'
    },
    {
        'Strategy': 'Covered Call',
        'Initial Cost': f"${S_gld - covered_call_price:.2f}",
        'Max Loss': f"${S_gld - covered_call_price:.2f}",
        'Max Profit': f"${K_covered - S_gld + covered_call_price:.2f}",
        'Breakeven': f"${S_gld - covered_call_price:.2f}",
        'Best For': 'Income generation',
        'Risk Level': '⭐⭐'
    },
    {
        'Strategy': 'Long Straddle',
        'Initial Cost': f"${straddle_cost:.2f}",
        'Max Loss': f"${straddle_cost:.2f}",
        'Max Profit': 'Unlimited',
        'Breakeven': f"${K_straddle:.2f}±${straddle_cost:.2f}",
        'Best For': 'High volatility',
        'Risk Level': '⭐⭐⭐⭐'
    }
])

print("\n📊 Options Strategies Comparison:\n")
display(strategies_comparison)

print("\n⭐ Risk Levels: ⭐ = Low, ⭐⭐⭐⭐⭐ = Very High")

## 🎯 Real-World Application: Complete Trade Example

Let's simulate a complete trade using the GLD-GDX pair with options enhancement.

In [ ]:
# Current spread position
current_spread = spread.iloc[-1]
mean_spread = spread.mean()
std_spread = spread.std()
z_score = (current_spread - mean_spread) / std_spread

print(f"\n🎯 Current Market Conditions:\n")
print(f"Current Spread: {current_spread:.2f}")
print(f"Mean Spread: {mean_spread:.2f}")
print(f"Std Dev: {std_spread:.2f}")
print(f"Z-Score: {z_score:.2f}")
print(f"\n📍 Signal: ", end="")

if z_score < -1:
    print("🟢 BUY (spread below mean)")
    signal = 'BUY'
elif z_score > 1:
    print("🔴 SELL (spread above mean)")
    signal = 'SELL'
else:
    print("🟡 NEUTRAL (wait for better entry)")
    signal = 'NEUTRAL'

In [ ]:
# Construct trade with options
if signal == 'BUY':
    print(f"\n📋 Recommended Trade Structure:\n")
    print(f"Base Strategy:")
    print(f"  1. Buy GLD shares: ${S_gld:.2f} each")
    print(f"  2. Short GDX shares: ${data['GDX'].iloc[-1]:.2f} each (hedge ratio: {beta:.4f})")
    print(f"\nOptions Enhancement:")
    print(f"  3. Buy ATM call on GLD @ ${call_price:.2f} (leverage)")
    print(f"  4. Buy 5% OTM put on GLD @ ${put_price:.2f} (protection)")
    print(f"\nTotal Capital Required:")
    
    base_capital = 10000  # Example
    gld_shares = base_capital / S_gld
    gdx_shares = gld_shares * beta
    
    options_cost = call_price + put_price
    n_contracts = int(gld_shares / 100)
    total_options_cost = options_cost * n_contracts * 100
    
    print(f"  Base Position: ${base_capital:.2f}")
    print(f"  Options Cost: ${total_options_cost:.2f} ({n_contracts} contracts)")
    print(f"  Total: ${base_capital + total_options_cost:.2f}")
    
    print(f"\n📊 Position Summary:")
    print(f"  GLD shares: {gld_shares:.0f}")
    print(f"  GDX shares (short): {gdx_shares:.0f}")
    print(f"  GLD calls: {n_contracts} contracts")
    print(f"  GLD puts: {n_contracts} contracts")
    
    print(f"\n🎯 Targets & Stops:")
    print(f"  Take Profit: Spread crosses ${mean_spread:.2f} (mean)")
    print(f"  Stop Loss: Spread falls below ${mean_spread - 2*std_spread:.2f} (-2σ)")
    print(f"  Options Expiration: 30 days")
    
    print(f"\n💰 Expected Outcomes:")
    target_return = std_spread / S_gld
    base_profit = base_capital * target_return
    leveraged_profit = base_profit * 2  # With options leverage
    
    print(f"  Base Strategy: ${base_profit:.2f} ({target_return*100:.2f}%)")
    print(f"  With Options: ${leveraged_profit:.2f} ({target_return*200:.2f}%)")
    print(f"  Max Protected Loss: ${(S_gld - K_put)*gld_shares:.2f}")

elif signal == 'SELL':
    print(f"\n📋 Recommended Trade Structure:\n")
    print(f"Similar to BUY but inverted:")
    print(f"  - Short GLD, Long GDX")
    print(f"  - Buy ATM put options (for leverage)")
    print(f"  - Buy OTM call options (for protection)")
else:
    print(f"\n⏸️ No trade recommended. Wait for |Z-Score| > 1")

## 🎓 Key Lessons & Best Practices

### 1. Options Selection
- **Time to Expiration**: 30-60 days optimal for pairs trading
- **Strike Selection**: ATM for leverage, OTM for protection
- **Liquidity**: Always check bid-ask spreads

### 2. Risk Management
- **Position Sizing**: Never risk more than 2-3% per trade
- **Greeks Monitoring**: Watch delta, gamma, theta daily
- **Stop Losses**: Set based on statistical thresholds

### 3. When to Use Options
- ✅ **High conviction trades** (strong statistical signals)
- ✅ **Limited capital** (leverage efficiency)
- ✅ **Risk-defined trades** (known max loss)
- ❌ **Low volatility** environments (expensive relative to potential)
- ❌ **Tight spreads** (options cost eats into profit)

### 4. Common Pitfalls
- ⚠️ **Overpaying for options** (check implied vs historical volatility)
- ⚠️ **Ignoring theta decay** (time is your enemy)
- ⚠️ **Over-leveraging** (options magnify losses too)
- ⚠️ **Not adjusting positions** (delta changes as prices move)

---

## 🚀 Advanced Topics for Further Study

1. **Implied Volatility Surface**: Understanding vol skew and term structure
2. **Dynamic Hedging**: Adjusting delta throughout trade life
3. **Volatility Arbitrage**: Trading realized vs implied volatility
4. **Options on Spreads**: Synthetic spread options construction
5. **Greeks Management**: Portfolio-level risk optimization
6. **Machine Learning**: Predicting option profitability

---

## ⚠️ Disclaimer

This notebook is for **educational purposes only**. Options trading involves significant risk and is not suitable for all investors. Past performance does not guarantee future results. Always:
- Paper trade strategies before using real money
- Understand your risk tolerance
- Consider consulting a financial advisor
- Never trade with money you cannot afford to lose